In [0]:
%pip install tabulate
%restart_python


In [0]:
from databricks.sdk import WorkspaceClient
from tabulate import tabulate
import time

In [0]:
space_id = dbutils.widgets.get("genie-space-id")
print(space_id)


In [0]:
w = WorkspaceClient()

In [0]:
print(w.genie.get_space(space_id=space_id))

In [0]:
# Start a new conversation — ask a question in natural language
question = "Who are the top 10 suppliers in AMER. Summarize the response in natural language."


### Start Conversation and Wait - Blocking

In [0]:
# This method sends the request and waits until the reply (or query) is ready
message = w.genie.start_conversation_and_wait(
    space_id = space_id,
    content = question
)

# The returned object is a GenieMessage (per current SDK behavior) — it contains message metadata and attachments
print("Message metadata:", message)


In [0]:
# The 'attachments' field may include a query (SQL) result or other response types
if message.attachments:
    for att in message.attachments:
        # For example, if Genie generated a SQL query / result
        if att.query:
            print("Generated SQL:", att.query.query)
            print("Query description:", att.query.description)

            # If the query executed and returned results, you can inspect metadata
            if att.query.query_result_metadata:
                print("\nResult metadata:", att.query.query_result_metadata)
                print("\n=== Full Query Result ===")
                full_result = w.genie.get_message_attachment_query_result(
                    space_id=space_id,
                    conversation_id=message.conversation_id,
                    message_id=message.id,
                    attachment_id=att.attachment_id
                )
                # print("\n=== Full Result ===")
                # print(full_result)
                columns = [col.name for col in full_result.statement_response.manifest.schema.columns]
                rows = full_result.statement_response.result.data_array
                print(tabulate(rows, headers=columns, tablefmt="fancy_grid"))
else:
    print("No attachments in response.")

### Start Conversation - Non Blocking

In [0]:
# This immediately returns the initial message
message = w.genie.start_conversation(
    space_id=space_id,
    content=question
)

conversation_id = message.conversation_id
message_id = message.message_id
response = message.response
print("Conversation ID:", conversation_id)
print("Message ID:", message_id)
print("Message response:", response)

In [0]:
POLL_INTERVAL = 2
MAX_WAIT = 60
reply_message = None
start_time = time.time()

while time.time() - start_time < MAX_WAIT:
    msg = w.genie.get_message(
        space_id=space_id,
        conversation_id=conversation_id,
        message_id=message_id
    )
    #print(f"----------\n{msg}\n-----------\n")
    # Check message status
    if msg.status.name == "COMPLETED":
        reply_message = msg
        break
    elif msg.status.name == "FAILED":
        raise RuntimeError(f"Genie failed: {msg.status.error}")
    
    time.sleep(POLL_INTERVAL)

if not reply_message:
    raise TimeoutError("Timed out waiting for Genie reply.")

print("\n=== Genie Message Metadata ===")
print(reply_message)




In [0]:
if reply_message.attachments:
    for att in reply_message.attachments:
        if att.query:
            print("\n=== Generated SQL ===")
            print(att.query.query)

            print("\n=== Query Description ===")
            print(att.query.description)

            if att.query.query_result_metadata:
                print("\n=== Result Metadata ===")
                print(att.query.query_result_metadata)

            # -------------------------------
            # Deprecated: fetch the full query result
            # -------------------------------
            full_result = w.genie.get_message_attachment_query_result(
                space_id=space_id,
                conversation_id=conversation_id,
                message_id=reply_message.id,
                attachment_id=att.attachment_id
            )

            # Extract column names and rows from the full_result object
            columns = [col.name for col in full_result.statement_response.manifest.schema.columns]
            rows = full_result.statement_response.result.data_array

            # Pretty-print the query result
            print("\n=== Full Query Result ===")
            print(tabulate(rows, headers=columns, tablefmt="fancy_grid"))
else:
    print("\nNo attachments in response.")

### Follow Ups - With create_message() - Non-Blocking

In [0]:
follow_up_question='What are the top 10 suppliers in the UNITED STATES by account balance?'

In [0]:
waiter = w.genie.create_message(
    space_id=space_id,
    conversation_id=conversation_id,
    content=follow_up_question
)

In [0]:
POLL_INTERVAL = 2
MAX_WAIT = 60
follow_reply = None
start_time = time.time()

while time.time() - start_time < MAX_WAIT:
    msg = w.genie.get_message(
        space_id=space_id,
        conversation_id=conversation_id,
        message_id=waiter.message_id
    )

    if msg.status.name == "COMPLETED":
        follow_reply = msg
        break
    elif msg.status.name == "FAILED":
        raise RuntimeError(f"Genie failed to process the follow-up message: {msg.status.error}")

    time.sleep(POLL_INTERVAL)

if not follow_reply:
    raise TimeoutError("Timed out waiting for follow-up reply.")

print("\n=== Follow-up Message Metadata ===")
print(follow_reply)



In [0]:
# Process attachments and print full query results
if follow_reply.attachments:
    for att in follow_reply.attachments:
        if att.query:
            print("\n=== Follow-up Generated SQL ===")
            print(att.query.query)

            print("\n=== Query Description ===")
            print(att.query.description)

            # Full query result
            full_result = w.genie.get_message_attachment_query_result(
                space_id=space_id,
                conversation_id=conversation_id,
                message_id=follow_reply.id,
                attachment_id=att.attachment_id
            )

            columns = [col.name for col in full_result.statement_response.manifest.schema.columns]
            rows = full_result.statement_response.result.data_array
            print("\n=== Full Follow-up Query Result ===")
            print(tabulate(rows, headers=columns, tablefmt="fancy_grid"))
else:
    print("No attachments in follow-up response.")


### Follow Ups - With create_message_and_wait() - Blocking

In [0]:
follow_reply = w.genie.create_message_and_wait(
    space_id=space_id,
    conversation_id=conversation_id,
    content=follow_up_question
)

print("\n=== Follow-up Message Metadata ===")
print(follow_reply)


In [0]:
# Process attachments and print full query results
if follow_reply.attachments:
    for att in follow_reply.attachments:
        if att.query:
            print("\n=== Follow-up Generated SQL ===")
            print(att.query.query)

            print("\n=== Query Description ===")
            print(att.query.description)

            # Full query result
            full_result = w.genie.get_message_attachment_query_result(
                space_id=space_id,
                conversation_id=conversation_id,
                message_id=follow_reply.id,
                attachment_id=att.attachment_id
            )

            columns = [col.name for col in full_result.statement_response.manifest.schema.columns]
            rows = full_result.statement_response.result.data_array
            print("\n=== Full Follow-up Query Result ===")
            print(tabulate(rows, headers=columns, tablefmt="fancy_grid"))
else:
    print("No attachments in follow-up response.")


### With Exponential backoff

In [0]:
# This immediately returns the initial message
message = w.genie.start_conversation(
    space_id=space_id,
    content=question
)

conversation_id = message.conversation_id
message_id = message.message_id
response = message.response

print("Conversation ID:", conversation_id)
print("Message ID:", message_id)
print("Message response:", response)

In [0]:
# -------------------------------
# Polling with exponential backoff
# -------------------------------
reply_message = None
start_time = time.time()
timeout_seconds = 600   # 10 minutes max
poll_interval = 1       # initial poll interval in seconds
max_interval = 60       # max backoff interval

while time.time() - start_time < timeout_seconds:
    msg = w.genie.get_message(
        space_id=space_id,
        conversation_id=conversation_id,
        message_id=message_id
    )

    # Check status
    if msg.status.name in ["COMPLETED", "FAILED", "CANCELLED"]:
        reply_message = msg
        print(time.time() - start_time)
        break

    # Sleep and apply exponential backoff
    time.sleep(poll_interval)
    poll_interval = min(poll_interval * 2, max_interval)

if not reply_message:
    raise TimeoutError("Timed out waiting for Genie reply (10 minutes).")

# -------------------------------
# Print message metadata
# -------------------------------
print("\n=== Genie Message Metadata ===")
print(reply_message)

In [0]:
# -------------------------------
# Process attachments and full query result
# -------------------------------
if reply_message.attachments:
    for att in reply_message.attachments:
        if att.query:
            print("\n=== Generated SQL ===")
            print(att.query.query)

            print("\n=== Query Description ===")
            print(att.query.description)

            if att.query.query_result_metadata:
                print("\n=== Result Metadata ===")
                print(att.query.query_result_metadata)

            # Deprecated: fetch the full query result
            full_result = w.genie.get_message_attachment_query_result(
                space_id=space_id,
                conversation_id=conversation_id,
                message_id=reply_message.id,
                attachment_id=att.attachment_id
            )

            # Extract column names and rows
            columns = [col.name for col in full_result.statement_response.manifest.schema.columns]
            rows = full_result.statement_response.result.data_array

            # Pretty-print the table
            print("\n=== Full Query Result ===")
            print(tabulate(rows, headers=columns, tablefmt="fancy_grid"))
else:
    print("\nNo attachments in response.")
